In [1]:
import os
import polars as pl

In [2]:
pl.Config.set_tbl_rows(65)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)
pl.Config.set_verbose(True)  

polars.config.Config

In [3]:
slovenske = [
    "S SK 1939",
    "S SR 1939-1945"
]

In [4]:
projevy = pl.read_ndjson(
    'data_raw/schuze/prepis_*.ndjson', 
    ignore_errors=True, 
    schema=pl.Schema(
        {
            'mluvci': pl.String, 
            'soubor': pl.String, 
            'mluvci_id': pl.String, 
            'text': pl.String, 
            'poradi': pl.Int32}
    ))

_init_credential_provider_builder(): credential_provider_init = None
_init_credential_provider_builder(): credential_provider_init = None


In [ ]:
meta = pl.read_csv(
    'data_raw/schuze/meta_*.csv',
    schema=pl.Schema(
        {
            'komora': pl.String,
            'obdobi': pl.Int32,
            'schuze': pl.Int32,
            'cast': pl.Int32,
            'soubor': pl.String,
            'datum': pl.String,
            'komora_komplet': pl.String,
            'schuze_komplet': pl.String,
            'prepsano': pl.Boolean,
            'autorizovano': pl.Boolean
        }
    )
).with_columns(
    pl.col("datum").str.to_date(format="%Y-%m-%d",exact=False)
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
)

In [ ]:
meta.filter(pl.col("prepsano") == False).write_parquet("data/neprepsane_schuze.parquet")

In [ ]:
df = pl.read_csv('data_raw/schuze/meta_1990snr_stenprot_023schuz_s023008.csv')

In [ ]:
df

In [ ]:
try:
    os.makedirs("data")
except:
    pass

In [ ]:
projevy.filter(pl.col('soubor') == '1998ps_stenprot_019schuz_s019364.htm')

In [ ]:
zvoleni = projevy.filter(pl.col("mluvci").str.contains("zvolení ")).select(pl.col('mluvci')).unique().to_series().to_list()

In [ ]:
pritomni = projevy.filter(pl.col("mluvci").str.contains("[Pp]řítomn")).select(pl.col('mluvci')).unique().to_series().to_list()

In [ ]:
nejsou_mluvci = [
    "Přítomni",
    "Za vládu České republiky",
    "Nahoru",
    "Dolů",
    "2",
    "76",
    "Omluveni poslanci",
    
] + zvoleni + pritomni

In [ ]:
nejsou_mluvci

In [ ]:
try:
    
    import pickle
    with open(os.path.join("testy","testy.pickle"), 'rb') as testy_import:
        testy = pickle.load(testy_import)
    
    for t in testy:
        vysledek = projevy.filter(
                ((pl.col('mluvci').str.contains(t['kdo'])) & (pl.col('text').str.contains(t['co'])) 
        ))
    
        if len(vysledek) != 1:
            print(f'❌ {t['kdy']}, {t['co']}')
            if t.get('problem'):
                print(f'({t['problem']})')
        else:
            print(f'✔️ {t['kdy']}, {t['co']}')

except:
    pass

Chyba zde ještě neznamená, že je něco špatně – některá data se vyplňují právě až při spojování a čištění.

In [ ]:
df = projevy.join(
    meta, on="soubor", how="left"
).filter(
    ~pl.col("mluvci").is_in(nejsou_mluvci, nulls_equal=True)
).filter(
    ~pl.col('komora_komplet').str.contains('SNR')
).filter(
    ~pl.col('komora_komplet').str.contains('NR SR')
).filter(
    ~pl.col('komora_komplet').is_in(slovenske)
).with_columns(
    pl.col("text").str.replace_many(["   ","  "],[" "])
).with_columns(
    pl.col("text").str.replace(r"\([Pp]okračuje .{3,30}\)","").str.strip_chars()              
).sort(
    by=["komora_komplet","schuze","cast","poradi"]
).with_columns(
    pl.when(pl.col('mluvci').is_null()).then(pl.lit(False)).otherwise(pl.lit(True)).alias('mluvci_jisty')
).with_columns(
    pl.col("mluvci").forward_fill()
).with_columns(
    # Create a helper column to track when mluvci changes
    (pl.col("mluvci") != pl.col("mluvci").shift(1)).alias("mluvci_changed")
).with_columns(
    # Forward fill mluvci_id only when mluvci hasn't changed
    pl.when(pl.col("mluvci_changed") | pl.col("mluvci_id").is_not_null())
    .then(pl.col("mluvci_id"))
    .otherwise(None)
    .forward_fill()
    .alias("mluvci_id")
).drop("mluvci_changed")

# .filter(
#    ~(pl.col("text").str.starts_with("(pokračuje") | pl.col("text").str.starts_with("(Pokračuje")) #  | pl.col("text").str.ends_with("hodin)") | pl.col("text").str.ends_with("hodin.)") 

In [ ]:
df.write_parquet("data/projevy.parquet", use_pyarrow=True)

In [ ]:
komory = df.select(pl.col("komora")).unique().to_series().to_list()

In [ ]:
df.select(pl.col(["komora_komplet","komora"])).unique().sort(by="komora_komplet")

In [ ]:
komory

In [ ]:
df.filter(pl.col("soubor") == "1998ps_stenprot_019schuz_s019364.htm").select(pl.col("text"))

In [ ]:
for k in komory:
    obdobi = df.filter(pl.col('komora') == k).select(pl.col('obdobi')).unique().to_series().to_list()
    for o in obdobi:
        df.filter(pl.col('komora') == k).filter(pl.col('obdobi') == o).sort(by=['datum','schuze','cast','poradi']).write_parquet(f"data/{k}_{o}.parquet")

In [ ]:
len(df)

In [ ]:
len(meta)

In [ ]:
bez_data = meta.filter(pl.col("datum").is_null())
print(len(bez_data))

In [ ]:
bez_data.filter(pl.col("prepsano") == True)

In [ ]:
bez_data.sort(by='obdobi').tail(30)

In [ ]:
bez_komory = meta.filter(pl.col("komora_komplet").is_null())
print(len(bez_komory))

In [ ]:
bez_obdobi = meta.filter(pl.col("obdobi").is_null())
print(len(bez_obdobi))

In [ ]:
bez_data = meta.filter(pl.col("datum").is_null()).sample(30)
print(len(bez_data))

In [ ]:
meta.filter(pl.col("datum").is_null()).sample(30)

In [ ]:
import altair as alt

In [ ]:
alt.Chart(
    df.filter(~pl.col('datum').is_null()).sort(by='datum').group_by_dynamic(index_column="datum", every="1y").agg(pl.col('text').len()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("text:Q")
)

In [ ]:
df.filter(pl.col('datum').dt.year() == 1976)